# Aula 09 - Exercício 08
## SVM
### Alunos:
- Alexandre Batistella Bellas, 9763168
- Moisés Botarro Ferraz Silva, 8504135


---
 ### Descrição do dataset
 
 
 O dataset "Heart Diseases " (mais informações [aqui](https://www.kaggle.com/ronitf/heart-disease-uci/version/1)) é um dataset de classificação que contém 13 atributos numéricos, que representam alguns índices médicos de pacientes. O atributo "target" refere-se a presença - ou não - de doênça cardíaca no paciente.
 
--- 
### Questão 01.
Carregue o arquivo `heart.csv`, separe os dados (variáveis independentes e dependente) e escale os valores entre 0 e 1.

In [1]:
import pandas as pd
import numpy as np
import random 

random.seed(1)

df = pd.read_csv('heart.csv')
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
# Separação entre variáveis dependentes e independentes

X = np.array(df[df.columns[:-1]], dtype='float')
Y = df[df.columns[-1]]

print('X shape: ', X.shape)
print(X, end='\n\n')

print('Y shape', Y.shape)
print(Y.head(5))

X shape:  (303, 13)
[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]

Y shape (303,)
0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [3]:
# Normalizar valores entre 0 e 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_norm = scaler.transform(X)

print('shape X_norm: ', X_norm.shape)
print(X_norm)

shape X_norm:  (303, 13)
[[0.70833333 1.         1.         ... 0.         0.         0.33333333]
 [0.16666667 1.         0.66666667 ... 0.         0.         0.66666667]
 [0.25       0.         0.33333333 ... 1.         0.         0.66666667]
 ...
 [0.8125     1.         0.         ... 0.5        0.5        1.        ]
 [0.58333333 1.         0.         ... 0.5        0.25       1.        ]
 [0.58333333 0.         0.33333333 ... 0.5        0.25       0.66666667]]


--- 
### Questão 02.
Instancie diferentes configurações de SVM's:

* SVM linear
* SVM polinomial de ordem 2
* SVM polinomial de ordem 3
* SVM RBF com `gamma = 'auto'`


Para deixar o código mais organizável e legível, utilize estruturas do tipo dicionário para instanciar cada classificador bem como um array de scores. Por exemplo, para testar um classificador MLP com 1 camada escondida de 5 neuronios e outro com 2 camadas escondidas de 5 neurônios,  poderíamos instancia-los da forma:

```
experimentos = { 
    "MLP camada escondida (5,)": {
        "model" : MLPClassifier(hidden_layer_sizes=(5,),
        "scores" : []
    },
    "MLP camada escondida (5,5)": {
        "model" : MLPClassifier(hidden_layer_sizes=(5,5),
        "scores" : []
    }
}
```

In [4]:
from sklearn.svm import SVC

# Instanciação dos modelos

experimentos = {}
experimentos['SVM Linear'] = {'model': SVC(kernel='linear', gamma='auto'), 'scores':[]}
experimentos['SVM Polinomial de Ordem 2'] = {'model': SVC(kernel='poly', degree=2, gamma='auto'), 'scores':[]}
experimentos['SVM Polinomial de Ordem 3'] = {'model': SVC(kernel='poly', degree=3, gamma='auto'), 'scores':[]}
experimentos['SVM RBF com gamma = "auto"'] = {'model': SVC(kernel='rbf', gamma='auto'), 'scores':[]}

experimentos

{'SVM Linear': {'model': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'scores': []},
 'SVM Polinomial de Ordem 2': {'model': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'scores': []},
 'SVM Polinomial de Ordem 3': {'model': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'scores': []},
 'SVM RBF com gamma = "auto"': {'model': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=



---

### Questão 03.

Utilize 10-fold cross-validation no conjunto de dados para treinar e obter scores de acurácia dos classificadores para cada fold, de maneira genérica. Ou seja, para cada fold, sua rotina deve iterar pelo dicionário e treinar todos os classificadores e extrair uma medida de score, que será utilizada posteriormente. 

Para facilitar o debug, imprima informações como o número do fold atual e qual modelo está sendo treinado.

In [5]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)

it = 0
for train_index, test_index in skf.split(X_norm, Y):
    it += 1
    print("Fold número %d" % it)
    X_train, X_test = X_norm[train_index], X_norm[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    
    for model_name, model_obj in experimentos.items():
        print("\tTreinando Modelo %s" % model_name)
        
        model = model_obj['model']
        model.fit(X_train, Y_train)
        model_obj['scores'].append(model.score(X_test, Y_test))

Fold número 1
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial de Ordem 3
	Treinando Modelo SVM RBF com gamma = "auto"
Fold número 2
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial de Ordem 3
	Treinando Modelo SVM RBF com gamma = "auto"
Fold número 3
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial de Ordem 3
	Treinando Modelo SVM RBF com gamma = "auto"
Fold número 4
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial de Ordem 3
	Treinando Modelo SVM RBF com gamma = "auto"
Fold número 5
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial de Ordem 3
	Treinando Modelo SVM RBF com gamma = "auto"
Fold número 6
	Treinando Modelo SVM Linear
	Treinando Modelo SVM Polinomial de Ordem 2
	Treinando Modelo SVM Polinomial 



---

### Questão 04.

Finalmente, para cada classificador, exiba a acurácia média e seu desvio padrão. Com base nessas duas medidas, determine qual foi a melhor configuração testada.

In [6]:
for model_name, model_obj in experimentos.items():
    print(model_name)
    scores = model_obj['scores']
    print("\tscores: %s" % ["%.2f%%" % (s*100) for s in scores])
    print("\tmédia: %.2f%%" % (np.mean(scores)*100 ))
    print("\tdesvio padrão: %.2f%%" % (np.std(scores)*100 ))

SVM Linear
	scores: ['87.10%', '77.42%', '83.87%', '87.10%', '83.87%', '76.67%', '83.33%', '86.67%', '68.97%', '75.86%']
	média: 81.08%
	desvio padrão: 5.75%
SVM Polinomial de Ordem 2
	scores: ['80.65%', '70.97%', '80.65%', '77.42%', '74.19%', '76.67%', '86.67%', '76.67%', '72.41%', '75.86%']
	média: 77.21%
	desvio padrão: 4.32%
SVM Polinomial de Ordem 3
	scores: ['77.42%', '64.52%', '80.65%', '70.97%', '77.42%', '66.67%', '80.00%', '63.33%', '65.52%', '62.07%']
	média: 70.86%
	desvio padrão: 6.97%
SVM RBF com gamma = "auto"
	scores: ['83.87%', '80.65%', '83.87%', '90.32%', '83.87%', '83.33%', '80.00%', '76.67%', '72.41%', '86.21%']
	média: 82.12%
	desvio padrão: 4.75%


Comparando os resultados para média e desvio padrão obtidos para cada modelo, conclui-se que o SVM RBF com gamma automático foi a melhor configuração testada. 

Ela foi a que apresentou maior acurácia média (82.12%) e comparando-a com o modelo com segunda melhor média (SVM Linear: 81.08%), ela ainda possui menor desvio padrão (4.75% contra 5.75% do modelo linear).